In [1]:
import pandas as pd
import os
import re
from shutil import copyfile

In [2]:
#Read files
xls = pd.ExcelFile('BIDS_info.xlsx')

#path variables
BIDS='./BIDS_data/'
NIMS='./NIMS_data/'

#Make folder if folder doesn't exist function
def makefolder(name):
    if not os.path.exists(name):
        os.makedirs(name)

In [3]:
participants = xls.parse('participants')
protocol = xls.parse('protocol', convert_float=False).iloc[1:,:4] #columns 5 on are reference columns
protocol = protocol.dropna(axis=0, thresh=3) #get rid of items that don't have a bids equivalent
protocol.run_number = protocol.run_number.astype('str').str.strip('.0')
protocol["BIDS_scan_title_path"] = BIDS + "sub-###/" + protocol.ANAT_or_FUNCT + "/sub-###_" + protocol.BIDS_scan_title + "_run-" + protocol.run_number + ".nii.gz"

protocol.BIDS_scan_title_path = protocol.BIDS_scan_title_path.str.replace('_run-nan', '')

NIMS_dir_ref = protocol.NIMS_scan_title.tolist()

NIMS_BIDS_conversion = protocol[[0,4]]

In [ ]:
#Create Participants folders in BIDS if they don't exist
#makefolder(BIDS)
#participants.participant_id.apply(lambda x: makefolder(BIDS + 'sub-' + str(x) + "/anat"))
#participants.participant_id.apply(lambda x: makefolder(BIDS + 'sub-' + str(x) + "/funct"))

In [18]:
def check_against_protocol(participants,protocol): 
    
    all_files_correct = True

    for index, row in participants.iterrows():
        #Get files
        try:
            NIMS_dir_files = os.listdir(NIMS + row.nims_title)
            #Delete all non-nii.gz files
            NIMS_dir_files = [x for x in NIMS_dir_files if ".nii.gz"  in x]

            for item in set(NIMS_dir_ref):
                directory_filenames = [x for x in NIMS_dir_files if item in x]
                protocol_filenames = NIMS_BIDS_conversion[NIMS_BIDS_conversion.NIMS_scan_title.str.contains(item)]
                protocol_filenames = protocol_filenames.iloc[:,1].tolist()

                if len(directory_filenames) == len(protocol_filenames):
                    print("sub-" + str(row.participant_id) + ": ++ " + item.rjust(20) + " match")

                else:
                    print("sub-" + str(row.participant_id) + ": -- "+ item.rjust(20) + " files do not match protocol")
                    all_files_correct = False
            print("------------")
        except:
            all_files_correct = False
            print("sub-" + str(row.participant_id) + ": ERROR - folder is missing \n ------")

        
        
        
    print("All your folders match your protocol") if all_files_correct else print("Some folders do not match your protocol, please resolve errors")
    
    return all_files_correct

In [19]:
check_against_protocol(participants, protocol)

sub-100: ++   BOLD_EPI_29mm_2sec match
sub-100: ++        T1w_9mm_BRAVO match
------------
sub-101: ++   BOLD_EPI_29mm_2sec match
sub-101: ++        T1w_9mm_BRAVO match
------------
sub-102: ++   BOLD_EPI_29mm_2sec match
sub-102: ++        T1w_9mm_BRAVO match
------------
sub-103: ++   BOLD_EPI_29mm_2sec match
sub-103: ++        T1w_9mm_BRAVO match
------------
sub-104: ++   BOLD_EPI_29mm_2sec match
sub-104: ++        T1w_9mm_BRAVO match
------------
sub-105: ERROR - folder is missing 
 ------
Some folders do not match your protocol, please resolve errors


In [24]:
def rename_to_bids(participants, protocol):
    print("Comparing Folders to Protocol...\n")
    if check_against_protocol(participants,protocol):
        for index, row in participants.iterrows():
            #Get files
            NIMS_dir_files = os.listdir(NIMS + row.nims_title)

            #Delete all non-nii.gz files
            NIMS_dir_files = [x for x in NIMS_dir_files if ".nii.gz"  in x]

            for item in set(NIMS_dir_ref):
                directory_filenames = [x for x in NIMS_dir_files if item in x]
                protocol_filenames = NIMS_BIDS_conversion[NIMS_BIDS_conversion.NIMS_scan_title.str.contains(item)]
                protocol_filenames = protocol_filenames.iloc[:,1].tolist()

                assert len(directory_filenames) == len(protocol_filenames), "filelists aren't the same length"

                for index, item in enumerate(directory_filenames):
                    oldpath = (NIMS + row.nims_title + "/" + directory_filenames[index])
                    newpath = (protocol_filenames[index].replace("###", str(row.participant_id)))
                    copyfile(oldpath, newpath)

                    print("sub-" + str(row.participant_id) + item.rjust(20))
            print("------------")



In [25]:
rename_to_bids(participants, protocol)

Comparing Folders to Protocol...

sub-100: ++   BOLD_EPI_29mm_2sec match
sub-100: ++        T1w_9mm_BRAVO match
------------
sub-101: ++   BOLD_EPI_29mm_2sec match
sub-101: ++        T1w_9mm_BRAVO match
------------
sub-102: ++   BOLD_EPI_29mm_2sec match
sub-102: ++        T1w_9mm_BRAVO match
------------
sub-103: ++   BOLD_EPI_29mm_2sec match
sub-103: ++        T1w_9mm_BRAVO match
------------
sub-104: ++   BOLD_EPI_29mm_2sec match
sub-104: ++        T1w_9mm_BRAVO match
------------
sub-105: ERROR - folder is missing 
 ------
Some folders do not match your protocol, please resolve errors
